In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import seaborn as sns
iris = sns.load_dataset('iris')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
data = torch.tensor(iris[iris.columns[0:4]].values).float()

labels = torch.zeros(len(data),dtype = torch.long)

labels[iris.species=='versicolor']= 1
labels[iris.species=='virginica']= 2

labels

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])

In [4]:
class ANNiris(nn.Module):
    def __init__(self,nUnits,nLayers):
        super().__init__()
        
        self.layers = nn.ModuleDict()
        self.nLayers = nLayers
        
        self.layers['input'] = nn.Linear(4,nUnits)
        
        for i in range(nLayers):
            self.layers[f'hidden{i}'] = nn.Linear(nUnits,nUnits)
            
        self.layers['output'] = nn.Linear(nUnits,3)
        
    def forward(self,x):
        
        x = self.layers['input'](x)
        
        for i in range(self.nLayers):
            x = F.relu(self.layers[f'hidden{i}'](x))
        x = self.layers['output'](x)
            
        return x

In [5]:
nUnitsPerLayer = 12
nLayers = 4
net = ANNiris(nUnitsPerLayer,nLayers)
net

ANNiris(
  (layers): ModuleDict(
    (input): Linear(in_features=4, out_features=12, bias=True)
    (hidden0): Linear(in_features=12, out_features=12, bias=True)
    (hidden1): Linear(in_features=12, out_features=12, bias=True)
    (hidden2): Linear(in_features=12, out_features=12, bias=True)
    (hidden3): Linear(in_features=12, out_features=12, bias=True)
    (output): Linear(in_features=12, out_features=3, bias=True)
  )
)

In [6]:

tmpx = torch.randn(10,4)

y = net(tmpx)


In [1]:
def trainTheModel(theModel):
    
    lossfun = nn.CrossEntropyLoss()
    
    optimizer = torch.optim.SGD(theModel.parameters(),lr=.01)
    
    for epochi in range(numepochs):
        yHat = theModel(data)
        
        loss = lossfun(yHat,labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    predictions = theModel(data)
    predlabels = torch.argmax(predictions,axis=1)
    acc = 100*torch.mean((predslabels == labels).float())
    
    nParams = sum(p.numel() for p in theModel.parameters() if p.requires_grad)
    
    return acc,nParams